# 数位DP
- **数位DP的思想**: 我们需要在指定位数上填写数字, 每个数位可以填多个数字, 填完这些数字得到的整数需要满足某些条件, 常常问这样填数字的方案有多少?或者在满足某些情况时停止
- 数位DP的模板代码: 定义f(i, mask, isLimit, isNum) 表示构造第i位及其之后数位的合法方案数(i=0表示填写第0位及其以后数字的个数)。
    - mask: 由于此题要求每一个数字都不同因此我们使用mask表示已经选取了哪些数字. 
    - isLimit: 表示当前位是否有限制（true表示当前为需要有上界限制）
    - isNum: 表示在之前的位置上是否已经填了数字了（true表示前面已经有数字了）
- 对于递归我们需要使用@cache进行记忆化缓存, 因此我们的参数中不能含有集合这种元素, 哪如何去表示mask呢? 实际上数字集合与二进制数字是一一对应的, 00010: 表示数字1已经使用, 01001:表示数字1和数字3已经使用
    - 往集合中填一个数字d: 将 mask 更新为 mask | (1 << d)
    - 判断集合中是否含有数字d: mask >> d & 1 可以取出mask的第d个比特位，如果是1就说明d在集合中。
    - 往集合中删除一个数字d: 将mask 更新为 mask ^ (1 << d)

## 问题65：[统计特殊整数（数位DP）](https://leetcode.cn/problems/count-special-integers/description/)
### 题目
如果一个正整数每一个数位都是互不相同的，我们称它是特殊整数 。给你一个正整数n，请你返回区间[1, n]之间特殊整数的数目。

### 分析
- 需要用到模板代码中f(i, mask, isLimit, isNum)所有的入参
- isLimit: 在本题中假设我们的值要小于123, 如果当前面数位的值刚好为1 2 _, 那么我们填写第三位时只能填写[0, 3]范围, 但如果前面的值为1 0 _, 那么第三位的值可以是[0, 9]任意数, 因此我们需要用这个表示当前数位可以取哪些范围
- isNum: 我们需要用这个标识来帮我们确定, 当前数位是否可以填0, 如果前面并没有填写数字, 那么我们这一数位是不能从0开始填的。这个标识也可以帮助我们计算小于n的位数的情况(可以包含0001这种情况)


In [ ]:
from functools import cache

def process(n):
    s = str(n)  # 将n字符串化方便统计位数

    @cache
    def f(i, mask, isLimit, isNum):
        if i == len(s):   # 如果i已经到了最末尾
            # 返回1或者0, 如果isNum=False, 意味着前面都没有数字, 因此返回0, 否则返回1
            return int(isNum)  # 等价于 1 if isNum else 0

        res = 0  # 用于统计第i位置及其之后数位的合法方案数
        if isNum == False: # 表示第i位以前没有数字, 则i位置也可以选择跳过(需要加上这种情况)
            res += f(i+1, mask, False, False)     # 由于当前位选择了跳过, 则对下一位一定是没有限制的

        # 根据isLimit计算遍历的上界
        up = int(s[i]) if isLimit else 9    # 如果有限制, 则本位最大能选择s[i]
        # 根据isNum计算下界
        down = 0 if isNum else 1   # 如果前文已经有了数字了, 那么本位可以从0填写

        for d in range(down, up+1):
            if mask >> d & 1 == 0:  # 如果mask中没有d
                res += f(i+1, mask | (1 << d), isLimit and d == up, True)  # 如有要对后续数位有上界限制, 必须是之前所有数位都达到限制
        
        return res
    # 初始条件中需要使用isLimit=True是因为我们需要对第0位有上界限制
    return f(0, 0, True, False)